<a href="https://colab.research.google.com/github/Anish-Mukherjee/tomato-diseases/blob/main/tomato_diseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>